# Python: Static Panel Models with Fixed Effects

In this example, we illustrate how the [DoubleML](https://docs.doubleml.org/stable/index.html) package can be used to estimate treatment effects for static panel models with fixed effects in a partially linear panel regression [DoubleMLPLPR](https://docs.doubleml.org/stable/guide/models.html#partially-linear-models-plm) model. The model is based on [Clarke and Polselli (2025)](https://doi.org/10.1093/ectj/utaf011).

In [ ]:
import optuna
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.base import clone
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LassoCV
from lightgbm import LGBMRegressor

from doubleml.data import DoubleMLPanelData
from doubleml.plm.datasets import make_plpr_CP2025
from doubleml import DoubleMLPLPR

import warnings
warnings.filterwarnings("ignore")

## Data

We will use the implemented data generating process [make_plpr_CP2025](https://docs.doubleml.org/stable/api/datasets.html#dataset-generators) to generate data similar to the simulation in [Clarke and Polselli (2025)](https://doi.org/10.1093/ectj/utaf011). For exposition, we use the simple linear `dgp_type="dgp1"`, with 150 units, 10 time periods per unit, and a true treatment effect of `theta=0.5`.

We set `time_type="int"` such that the time variable values will be integers. It's also possible to use `"float"` or `"datetime"` time variables with [DoubleMLPLPR](https://docs.doubleml.org/stable/api/dml_models.html#doubleml-plm).

In [ ]:
np.random.seed(123)
data = make_plpr_CP2025(num_id=150, num_t=10, dim_x=30, theta=0.5, dgp_type="dgp1", time_type="int")
data.head()

To create a corresponding [DoubleMLPanelData](https://docs.doubleml.org/stable/api/generated/doubleml.data.DoubleMLPanelData.html) object, we need to set `static_panel=True` and specify `id_col` and `time_col` columns.

In [ ]:
data_obj = DoubleMLPanelData(data, y_col="y", d_cols="d", t_col="time", id_col="id", static_panel=True)

## Model

The partially linear panel regression (PLPR) model extends the partially linear model to panel data by introducing fixed effects $\alpha_i^*$.

The PLPR model takes the form

$$
\begin{align*}
    Y_{it} &= \theta_0 D_{it} + g_1(X_{it}) + \alpha_i^* + U_{it}, \\
    D_{it} &= m_1(X_{it}) + \gamma_i + V_{it},
\end{align*}
$$

where
- $Y_{it}$ outcome, $D_{it}$ treatment, $X_{it}$ covariates, $\theta_0$ causal treatment effect
- $g_1$ and $m_1$ nuisance functions
- $\alpha_i^*$, $\gamma_i$ unobserved individual heterogeneity, correlated with covariates
- $U_{it}$, $V_{it}$ error terms

Further note $\mathbb{E}[U_{it} \mid D_{it}, X_{it}, \alpha_i^*] = 0$ and $\mathbb{E}[V_{it} \mid X_{it}, \gamma_i]=0$, but $\mathbb{E}[\alpha_i^* \mid D_{it}, X_{it}] \neq 0$.

Alternatively we can write the partialling-out PLPR as 

$$
\begin{align*}
    Y_{it} &= \theta_0 V_{it} + \ell_1(X_{it}) + \alpha_i + U_{it}, \\
    V_{it} &= D_{it} - m_1(X_{it}) - \gamma_i,
\end{align*}
$$

with nuisance function $\ell_1$ and fixed effect $\alpha_i$.

### Assumptions

Define $\xi_i$ as time-invariant heterogeneity terms influencing outcome and treatment and $L_{t-1}(W_i) = \{ W_{i1}, \dots, W_{it-1} \}$ as lags of a random variable $W_{it}$ at wave $t$.

- *No feedback to predictors*
$$ X_{it} \perp L_{t-1} (Y_i, D_i) \mid L_{t-1} (X_i), \xi_i $$
- *Static panel*
$$ Y_{it}, D_{it} \perp L_{t-1} (Y_i, X_i, D_i) \mid X_{it}, \xi_i $$
- *Selection on observables and omitted time-invariant variables*
$$ Y_{it} (.) \perp D_{it} \mid X_{it}, \xi_i $$
- *Homogeneity and linearity of the treatment effect*
$$ \mathbb{E} [Y_{it}(d) - Y_{it}(0) \mid X_{it}, \xi_i] = d \theta_0 $$
- *Additive Separability*
$$
\begin{align*}
\mathbb{E} [Y_{it}(0) \mid X_{it}, \xi_i] &= g_1(X_{it}) + \alpha^*_i \quad \text{where } \alpha^*_i = \alpha^*(\xi_i), \\
\mathbb{E} [D_{it} \mid X_{it}, \xi_i] &= m_1(X_{it}) + \gamma_i \quad \text{where } \gamma_i = \gamma(\xi_i)
\end{align*}
$$

For more information, see [Clarke and Polselli (2025)](https://doi.org/10.1093/ectj/utaf011).

To estimate the causal effect, we can create a [DoubleMLPLPR](https://docs.doubleml.org/stable/api/dml_models.html#doubleml-plm) object. The model described in [Clarke and Polselli (2025)](https://doi.org/10.1093/ectj/utaf011) uses block-k-fold cross-fitting, where the entire time series of the sampled unit is allocated to one fold to allow for possible serial correlation within each unit which is common with panel data. Furthermore, cluster robust standard error are employed. [DoubleMLPLPR](https://docs.doubleml.org/stable/guide/models.html#partially-linear-models-plm) implements both aspects by using `id_col` as the cluster variable.

## Estimation Approaches

[Clarke and Polselli (2025)](https://doi.org/10.1093/ectj/utaf011) describes multiple estimation approaches, which can be set with the `approach` parameter. Depending on the type of `approach`, different data transformations are performed along the way.

### Correlated Random Effects

The correlated random effects (cre) approaches includes the a general approach (`cre_general`) and  an approach relying on normality assumptions (`cre_normal`).

#### `cre_general`

The `cre_general` approach:

- Learning $\ell_1$ from $\{ Y_{it}, X_{it}, \bar{X}_i : t=1,\dots, T \}_{i=1}^N$.
- First learning $\tilde{m}_1$ from $\{ D_{it}, X_{it}, \bar{X}_i : t=1,\dots, T \}_{i=1}^N$, with predictions $\hat{m}_{1,it} = \tilde{m}_1 (X_{it}, \bar{X}_i)$
    - Calculate $\hat{\bar{m}}_i = T^{-1} \sum_{t=1}^T \hat{m}_{1,it}$,
    - Calculate final nuisance part as $\hat{m}^*_1 (X_{it}, \bar{X}_i, \bar{D}_i) = \hat{m}_{1,it} + \bar{D}_i - \hat{\bar{m}}_i$.

In [ ]:
learner = LassoCV()
ml_l = clone(learner)
ml_m = clone(learner)

dml_plpr_cre_general = DoubleMLPLPR(data_obj, ml_l=ml_l, ml_m=ml_m, approach="cre_general", n_folds=5)

We can look at the the transformed data using the `data_transform` property after the `DoubleMLPLPR` object was created.

In [ ]:
dml_plpr_cre_general.data_transform.data.head()

We can see that the covariates inlcude the original $X_{it}$ and additionally the unit mean $\bar{X}_i$.

After fitting the model, we can print the `DoubleMLPLPR` object. The data summary corresponds to the transformed data. Additional Information at the end also includes a pre-transformation data summary.

In [ ]:
dml_plpr_cre_general.fit()
print(dml_plpr_cre_general)

#### `cre_normal`

The `cre_normal` approach:

Under the assumption that the conditional distribution $D_{i1}, \dots, D_{iT} \mid X_{i1}, \dots X_{iT}$ is multivariate normal (see [Clarke and Polselli (2025)](https://doi.org/10.1093/ectj/utaf011) for further details):
- Learn $\ell_1$ from $\{ Y_{it}, X_{it}, \bar{X}_i : t=1,\dots, T \}_{i=1}^N$,
- Learn $m^*_{1}$ from $\{ D_{it}, X_{it}, \bar{X}_i, \bar{D}_i: t=1,\dots, T \}_{i=1}^N$.

In [ ]:
dml_plpr_cre_normal = DoubleMLPLPR(data_obj, ml_l=ml_l, ml_m=ml_m, approach="cre_normal", n_folds=5)
dml_plpr_cre_normal.fit()
print(dml_plpr_cre_normal.summary)

The `cre_normal` approach uses additionally inlcudes $\bar{D}_i$ in the treatment nuisance estimation. The corresponding data can be assesses by the `d_mean` property.

In [ ]:
dml_plpr_cre_normal.d_mean

### Transformation Approaches

The transformation approaches include first differences (`fd_exact`) and within-group (`wg_approx`) transformations.

#### `fd_exact`

Consider first differences (FD) transformation (`fd_exact`) $Q(Y_{it})= Y_{it} - Y_{it-1}$, under the assumptions from above, [Clarke and Polselli (2025)](https://doi.org/10.1093/ectj/utaf011) show that $\mathbb{E}[Y_{it}-Y_{it-1} | X_{it-1},X_{it}] =\Delta \ell_1 (X_{it-1}, X_{it})$ and $\mathbb{E}[D_{it}-D_{it-1} | X_{it-1},X_{it}] =\Delta m_1 (X_{it-1}, X_{it})$. Therefore, the transformed nuisance function can be learnt as

- $\Delta \ell_1 (X_{it-1}, X_{it})$ from $\{ Y_{it}-Y_{it-1}, X_{it-1}, X_{it} : t=2, \dots , T \}_{i=1}^N$,
- $\Delta m_1 (X_{it-1}, X_{it})$ from $\{ D_{it}-D_{it-1}, X_{it-1}, X_{it} : t=2, \dots , T \}_{i=1}^N$.

In [ ]:
dml_plpr_fd_exact = DoubleMLPLPR(data_obj, ml_l=ml_l, ml_m=ml_m, approach="fd_exact", n_folds=5)
dml_plpr_fd_exact.data_transform.data.head()

We see that the outcome and treatment variables are now labeled `y_diff` and `d_diff` to indicate the first-difference transformation. Moreover, lagged covariates $X_{it-1}$ are added and rows for the first time period are dropped.

In [ ]:
dml_plpr_fd_exact.fit()
print(dml_plpr_fd_exact.summary)

#### `wg_approx`

For within-group (WG) transformation (`wg_approx`) $Q(X_{it})= X_{it} - \bar{X}_{i}$, where $\bar{X}_{i} = T^{-1} \sum_{t=1}^T X_{it}$, approximate the model as

$$
\begin{align*}
    Q(Y_{it}) &\approx \theta_0 Q(D_{it}) + g_1 (Q(X_{it})) + Q(U_{it}), \\
    Q(D_{it}) &\approx m_1 (Q(X_{it})) + Q(V_{it}).
\end{align*}
$$

Similarly for the partialling-out PLPR

$$
Q(Y_{it}) \approx \theta_0 Q(V_{it}) + \ell_1 (Q(X_{it})) + Q(U_{it}).
$$

- $\ell_1$ can be learnt from transformed data $\{ Q(Y_{it}), Q(X_{it}) : t=1,\dots,T \}_{i=1}^N$,
- $m_1$ can be learnt from transformed data $\{ Q(D_{it}), Q(X_{it}) : t=1,\dots,T \}_{i=1}^N$.

In [ ]:
dml_plpr_wg_approx = DoubleMLPLPR(data_obj, ml_l=ml_l, ml_m=ml_m, approach="wg_approx", n_folds=5)
dml_plpr_wg_approx.data_transform.data.head()

We see that the outcome, treatment and covariate variables are now labeled `y_deman`, `d_deman`, `xi_deman` to indicate the within-group transformations.

In [ ]:
dml_plpr_wg_approx.fit()
print(dml_plpr_wg_approx.summary)

For the simple linear data generating process `dgp_type="dgp1"`, we can see that all approaches lead to estimated close the true effect of `theta=0.5`.

The `data_original` property additionally includes the original data before any transformation was applied.

In [ ]:
dml_plpr_wg_approx.data_original.data.head()

## Feature preprocessing pipelines

We can incorporate preprocessing pipelines. For example, when using Lasso, we may want to include polynomial and interaction terms. Here, we create a class that allows us to include, for example, polynomials of order 3 and interactions between all variables.

In [ ]:
class PolyPlus(BaseEstimator, TransformerMixin):
    """PolynomialFeatures(degree=k) and additional terms x_i^(k+1)."""

    def __init__(self, degree=2, interaction_only=False, include_bias=False):
        self.degree = degree
        self.extra_degree = degree + 1
        self.interaction_only = interaction_only
        self.include_bias = include_bias
        self.poly = PolynomialFeatures(degree=degree, interaction_only=interaction_only, include_bias=include_bias)

    def fit(self, X, y=None):
        self.poly.fit(X)
        self.n_features_in_ = X.shape[1]
        return self

    def transform(self, X):
        X = np.asarray(X)
        X_poly = self.poly.transform(X)
        X_extra = X ** self.extra_degree
        return np.hstack([X_poly, X_extra])

    def get_feature_names_out(self, input_features=None):
        input_features = np.array(
            input_features
            if input_features is not None
            else [f"x{i}" for i in range(self.n_features_in_)]
        )
        poly_names = self.poly.get_feature_names_out(input_features)
        extra_names = [f"{name}^{self.extra_degree}" for name in input_features]
        return np.concatenate([poly_names, extra_names])

For this example we use the non-linear and discontinuous `dgp_type="dgp3"`, with 30 covariates and a true treatment effect `theta=0.5`.

In [ ]:
dim_x = 30
theta = 0.5

np.random.seed(123)
data_dgp3 = make_plpr_CP2025(num_id=500, num_t=10, dim_x=dim_x, theta=theta, dgp_type="dgp3")
dml_data_dgp3 = DoubleMLPanelData(data_dgp3, y_col="y", d_cols="d", t_col="time", id_col="id", static_panel=True)

We can apply the polynomial and intercation transformation for specific sets of covariates. For example, for the `fd_exact` approach, we can apply it to the original $X_{it}$ and lags $X_{it-1}$ seperately using `ColumnTransformer`.

To achieve this, we pass need to pass the corresponding indices for these two sets. `DoubleMLPLPR` stacks sets $X_{it}$ and $X_{it-1}$ column-wise. Given our example data has 30 covariates, this means that the first 30 features in the nuisance estimation correspond to the original $X_{it}$, and the last 30 correspond to lags $X_{it-1}$. Therefore we define the indices `indices_x` and `indices_x_tr` as below.

In [ ]:
indices_x = [x for x in range(dim_x)]
indices_x_tr = [x + dim_x for x in indices_x]

preprocessor = ColumnTransformer(
    [
        (
            "poly_x",
            PolyPlus(degree=2, include_bias=False, interaction_only=False),
            indices_x,
        ),
        (
            "poly_x_tr",
            PolyPlus(degree=2, include_bias=False, interaction_only=False),
            indices_x_tr,
        ),
    ],
    remainder="passthrough",
)

This preprocessor can be applied for approaches `cre_general` and `cre_normal` in the same fashion. In this case the two sets of covariates would be the original $X_{it}$ and the unit mean $\bar{X}_i$.

**Remark**: Note that we set `remainder="passthrough"` such that all remaining features, not part of `indices_x` and `indices_x_tr`, would not be preprocessed but still included in the nuisance estimation. This is particularly important for the `cre_normal` approach, as $\bar{D}_i$ is further added to $X_{it}$ and $\bar{X}_i$ in the treatment nuisance model.

Finally, we can create the learner using a pipeline and fit the model.

In [ ]:
ml_lasso = make_pipeline(
    preprocessor, StandardScaler(), LassoCV(alphas=20, cv=2, n_jobs=5)
)

ml_lasso

In [ ]:
plpr_lasso_fd = DoubleMLPLPR(dml_data_dgp3, clone(ml_lasso), clone(ml_lasso), approach="fd_exact", n_folds=5)
plpr_lasso_fd.fit(store_models=True)
print(plpr_lasso_fd.summary)

Given that we apply the polynomial and interactions preprossing to two sets of 30 columns each, the number of features is 1050.

In [ ]:
plpr_lasso_fd.models["ml_m"]["d_diff"][0][0].named_steps["lassocv"].n_features_in_

As describes above, for the `cre_normal` approach adds $\bar{X}_i$ to the features used in the treatment nuisance estimation.

In [ ]:
plpr_lasso_cre_normal = DoubleMLPLPR(dml_data_dgp3, clone(ml_lasso), clone(ml_lasso), approach="cre_normal", n_folds=5)
plpr_lasso_cre_normal.fit(store_models=True)
print(plpr_lasso_cre_normal.summary)
plpr_lasso_cre_normal.models["ml_m"]["d"][0][0].named_steps["lassocv"].n_features_in_

For the `wg_approx` approach, there is only one set of features. We can create a similar learner for this setting.

In [ ]:
preprocessor_wg = ColumnTransformer(
    [
        (
            "poly_x",
            PolyPlus(degree=2, include_bias=False, interaction_only=False),
            indices_x,
        )
    ],
    remainder="passthrough",
)

ml_lasso_wg = make_pipeline(
    preprocessor_wg, StandardScaler(), LassoCV(alphas=20, cv=2, n_jobs=5)
)

In [ ]:
plpr_lasso_wg = DoubleMLPLPR(dml_data_dgp3, clone(ml_lasso_wg), clone(ml_lasso_wg), approach="wg_approx", n_folds=5)
plpr_lasso_wg.fit(store_models=True)
print(plpr_lasso_wg.summary)
plpr_lasso_wg.models["ml_l"]["d_demean"][0][0].named_steps["lassocv"].n_features_in_

We can see that for the more complicated data generating process `dgp3`, the approximation approach performs worse compared to the other approaches.

As another example, below we should how to select a specific covariate subset for preprocessing. This can be useful in case of the data includes dummy covariates, where adding polynomials might not be appropriate.

In [ ]:
x_cols = dml_data_dgp3.x_cols 
x_cols_to_pre = ["x3", "x6", "x22"]

indices_x_pre = [i for i, c in enumerate(x_cols) if c in x_cols_to_pre]

preprocessor_alt = ColumnTransformer(
    [
        (
            "poly_x",
            PolyPlus(degree=2, include_bias=False, interaction_only=False),
            indices_x_pre,
        )
    ],
    remainder="passthrough",
)
ml_lasso_alt = make_pipeline(
    preprocessor_alt, StandardScaler(), LassoCV(alphas=20, cv=2, n_jobs=5)
)

plpr_lasso_wg.learner["ml_l"] = ml_lasso_alt
plpr_lasso_wg.learner["ml_m"] = ml_lasso_alt

plpr_lasso_wg.fit(store_models=True)
plpr_lasso_wg.models["ml_l"]["d_demean"][0][0].named_steps["lassocv"].n_features_in_

In [ ]:
plpr_lasso_wg.models["ml_l"]["d_demean"][0][0].named_steps['columntransformer']

We can also look at the resulting features.

**Remark**: Note, however, that the feature names here refer only to the corresponding `x_cols` indices, not the column names from the `pd.DataFrame` because [DoubleML](https://docs.doubleml.org/stable/index.html) uses  `np.array`'s for fitting the model. Therefore the difference to the names from `x_cols_to_pre`.

In [ ]:
plpr_lasso_wg.models["ml_l"]["d_demean"][0][0].named_steps['columntransformer'].get_feature_names_out()


## Hyperparameter tuning

In this section we will use the `tune_ml_models()` method to tune hyperparameters using the [Optuna](https://optuna.org/) package. More details can found in the [Python: Hyperparametertuning with Optuna](https://docs.doubleml.org/stable/examples/learners/py_optuna.html) example notebook.

As an example, we use [LightGBM](https://lightgbm.readthedocs.io/en/stable/) regressors and compare the estimates for the different static panel model approaches, when applied to the non-linear and discontinuous `dgp3`.

In [ ]:
dim_x = 30
theta = 0.5

np.random.seed(11)
data_tune = make_plpr_CP2025(num_id=4000, num_t=10, dim_x=dim_x, theta=theta, dgp_type="dgp3")
dml_data_tune = DoubleMLPanelData(data_tune, y_col="y", d_cols="d", t_col="time", id_col="id", static_panel=True)
ml_boost = LGBMRegressor(random_state=314, verbose=-1)

In [ ]:
# parameter space for both ml models
def ml_params(trial):
    return {
        "n_estimators": 100,
        "learning_rate": trial.suggest_float("learning_rate", 0.1, 0.4, log=True),
        "max_depth": trial.suggest_int("max_depth", 2, 10),
        "min_child_samples": trial.suggest_int("min_child_samples", 1, 5),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-2, 5, log=True),
    }

param_space = {
    "ml_l": ml_params,
    "ml_m": ml_params
}

optuna_settings = {
    "n_trials": 100,
    "show_progress_bar": True,
    "verbosity": optuna.logging.WARNING,  # Suppress Optuna logs
}

In [ ]:
plpr_tune_cre_general = DoubleMLPLPR(dml_data_tune, clone(ml_boost), clone(ml_boost), approach="cre_general", n_folds=5)

plpr_tune_cre_general.tune_ml_models(
    ml_param_space=param_space,
    optuna_settings=optuna_settings,
)

plpr_tune_cre_general.fit()
plpr_tune_cre_general.summary

0.509102

In [ ]:
plpr_tune_cre_normal = DoubleMLPLPR(dml_data_tune, clone(ml_boost), clone(ml_boost), approach="cre_normal", n_folds=5)

plpr_tune_cre_normal.tune_ml_models(
    ml_param_space=param_space,
    optuna_settings=optuna_settings,
)

plpr_tune_cre_normal.fit()
plpr_tune_cre_normal.summary

In [ ]:
plpr_tune_fd = DoubleMLPLPR(dml_data_tune, clone(ml_boost), clone(ml_boost), approach="fd_exact", n_folds=5)

plpr_tune_fd.tune_ml_models(
    ml_param_space=param_space,
    optuna_settings=optuna_settings,
)

plpr_tune_fd.fit()
plpr_tune_fd.summary

In [ ]:
plpr_tune_wg = DoubleMLPLPR(dml_data_tune, clone(ml_boost), clone(ml_boost), approach="wg_approx", n_folds=5)

plpr_tune_wg.tune_ml_models(
    ml_param_space=param_space,
    optuna_settings=optuna_settings,
)

plpr_tune_wg.fit()
plpr_tune_wg.summary

In [ ]:
palette = sns.color_palette("colorblind")

ci_cre_general = plpr_tune_cre_general.confint()
ci_cre_normal = plpr_tune_cre_normal.confint()
ci_fd = plpr_tune_fd.confint()
ci_wg = plpr_tune_wg.confint()

comparison_data = {
    "Model": ["cre_general", "cre_normal", "fd_exact", "wg_approx"],
    "theta": [plpr_tune_cre_general.coef[0], plpr_tune_cre_normal.coef[0], plpr_tune_fd.coef[0], plpr_tune_wg.coef[0]],
    "se": [plpr_tune_cre_general.se[0], plpr_tune_cre_normal.se[0], plpr_tune_fd.se[0], plpr_tune_wg.se[0]],
    "ci_lower": [ci_cre_general.iloc[0, 0], ci_cre_normal.iloc[0, 0], ci_fd.iloc[0, 0], ci_wg.iloc[0, 0]],
    "ci_upper": [ci_cre_general.iloc[0, 1], ci_cre_normal.iloc[0, 1], ci_fd.iloc[0, 1], ci_wg.iloc[0, 1]]
}
df_comparison = pd.DataFrame(comparison_data)

print(f"True treatment effect: {theta}\n")
print(df_comparison.to_string(index=False))

# Create comparison plot 
plt.figure(figsize=(12, 6))

for i in range(len(df_comparison)):
    plt.errorbar(i, df_comparison.loc[i, "theta"],
                 yerr=[[df_comparison.loc[i, "theta"] - df_comparison.loc[i, "ci_lower"]],
                       [df_comparison.loc[i, "ci_upper"] - df_comparison.loc[i, "theta"]]],
                 fmt='o', capsize=5, capthick=2, ecolor=palette[i], color=palette[i],
                 label=df_comparison.loc[i, "Model"], markersize=10, zorder=2)
plt.axhline(y=theta, color=palette[4], linestyle='--',
            linewidth=2, label="True effect", zorder=1)

plt.title("Comparison across DoubleMLPLPR approaches")
plt.ylabel("Coefficient Value")
plt.xticks(range(4), df_comparison["Model"], rotation=15, ha="right")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

We again see that the `wg_approx` leads to a biased estimate in the non-linear and discontinuous `dgp3` setting. The approaches `cre_general`, `cre_normal`, `fd_exact`, in combination with [LightGBM](https://lightgbm.readthedocs.io/en/stable/) regressors, tuned using the [Optuna](https://optuna.org/) package, lead to estimate close to the true treatment effect.

This is line with the simulation results in [Clarke and Polselli (2025)](https://doi.org/10.1093/ectj/utaf011), albeit only for only one dataset in this example.